In [23]:
import copy
import logging
import json
import random

from collections import OrderedDict
def jsonDefault(OrderedDict):
    return OrderedDict.__dict__

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

%matplotlib inline

DEBUG switch_backend Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [2]:
logging.basicConfig(format='%(levelname)s %(funcName)s %(message)s', level=logging.DEBUG)

In [3]:
def roll_dice(number, die_type, modifier):
    roll = 0
    for _ in range(number):
        roll = roll + random.randint(1, die_type)
    roll = roll + modifier
    logging.debug(f"{number}d{die_type}+{modifier} = {roll}")
    return roll

In [24]:
Modifier = {
    1: -5,
    2: -4,
    3: -4,
    4: -3,
    5: -3,    
    6: -2,
    7: -2,
    8: -1,
    9: -1,
    10: 0,
    11: 0,
    12: 1,
    13: 1,
    14: 2,
    15: 2,
    16: 3,
    17: 3,
    18: 4,
    19: 4,
    20: 5,
    21: 5,
    22: 6,
    23: 6,
    24: 7,
    25: 7,
    26: 8,
    27: 8,
    28: 9,
    29: 9,
    30: 10
}

class Attack:
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    def __init__(self, name="unnamed", to_hit_modifier=0, damage_dice_type=1, number_damage_dice=1,
                damage_modifier=-1, number_targets=1, attack_save_attribute="none",
                attack_save_difficulty=0, has_advantage=False):
        self.name = name
        self.to_hit_modifier = to_hit_modifier
        self.damage_dice_type = damage_dice_type
        self.number_damage_dice = number_damage_dice
        self.damage_modifier = damage_modifier
        self.number_targets = number_targets 
        self.attack_save_attribute = attack_save_attribute
        self.attack_save_difficulty = attack_save_difficulty
        self.has_advantage = has_advantage;
        
    def __repr__(self):
        return json.dumps(self, default=jsonDefault, indent=4)
    def add_record_as_data(self,_record):
        self.__dict__.update(_record.__dict__)
    def add_record_as_attr(self,_record):
        self.record = _record
    
    
class Ability_Scores:
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
        
    def __init__(self,strength=0,dexterity=0,constitution=0,intelligence=0,wisdom=0,charisma=0):
        self.strength = strength
        self.dexterity = dexterity
        self.constitution = constitution
        self.intelligence = intelligence
        self.wisdom = wisdom
        self.charisma = charisma

    def __repr__(self):
        return json.dumps(self, default=jsonDefault, indent=4)
    def add_record_as_data(self,_record):
        self.__dict__.update(_record.__dict__)
    def add_record_as_attr(self,_record):
        self.record = _record


class Actor:
    PLAYER = "player"
    MONSTER = "monster"
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    def __init__(self, name="unnamed", role="unknown", hit_points=0, armor_class=0, ability_scores=Ability_Scores(), attacks=[], initiative=0, initiative_modifier=0, targets=[]):
        self.name = name
        self.role = role
        self.hit_points = hit_points
        self.armor_class = armor_class
        self.ability_scores = ability_scores
        self.attacks = attacks
        self.initiative = initiative
        self.initiative_modifier = initiative_modifier;
        self.targets = targets
        
    def __repr__(self):
        return json.dumps(self, default=jsonDefault, indent=4)
    def add_record_as_data(self,_record):
        self.__dict__.update(_record.__dict__)
    def add_record_as_attr(self,_record):
        self.record = _record

class Combat:
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    def __init__(self, monsters=[], players=[]):
        self.alive = {"monsters":copy.deepcopy(monsters), "players":copy.deepcopy(players)}
        self.dead = {"monsters":[], "players":[]}
        self.stats = {"monsters":[], "players":[]}
        
        self.round = 0
        
    def __repr__(self):
        return json.dumps(self, default=jsonDefault, indent=4)
    def add_record_as_data(self,_record):
        self.__dict__.update(_record.__dict__)
    def add_record_as_attr(self,_record):
        self.record = _record


In [25]:
goblin_ability_scores = Ability_Scores(8,14,10,10,8,8);
goblin_attacks = [
    Attack(name="scimitar", to_hit_modifier=4, damage_dice_type=6, number_damage_dice=1,
          damage_modifier=2)
];
goblin = Actor(name="goblin1", role=Actor.MONSTER, hit_points=7, armor_class=15,
                    ability_scores=goblin_ability_scores,
                    attacks = goblin_attacks)
goblin2 = Actor(name="goblin2", role=Actor.MONSTER, hit_points=7, armor_class=15,
                    ability_scores=goblin_ability_scores,
                    attacks = goblin_attacks)
goblin3 = Actor(name="goblin3", role=Actor.MONSTER, hit_points=7, armor_class=15,
                    ability_scores=goblin_ability_scores,
                    attacks = goblin_attacks)
goblin4 = Actor(name="goblin4", role=Actor.MONSTER, hit_points=7, armor_class=15,
                    ability_scores=goblin_ability_scores,
                    attacks = goblin_attacks)


monsters = [goblin, goblin2, goblin3, goblin4];

In [26]:
character_ability_scores = Ability_Scores(17, 14, 16, 7, 11, 8);
character_attacks = [
    Attack(name="Greataxe", to_hit_modifier=5, damage_dice_type=12, number_damage_dice=1,
          damage_modifier=3)
];
galdric = Actor(name="Galdric", role=Actor.PLAYER, hit_points=15, armor_class=15,
                    ability_scores=character_ability_scores,
                    attacks = character_attacks)
larry = Actor(name="Larry", role=Actor.PLAYER, hit_points=15, armor_class=15,
                    ability_scores=character_ability_scores,
                    attacks = character_attacks)
curley = Actor(name="Curley", role=Actor.PLAYER, hit_points=15, armor_class=15,
                    ability_scores=character_ability_scores,
                    attacks = character_attacks)
moe = Actor(name="Moe", role=Actor.PLAYER, hit_points=15, armor_class=15,
                    ability_scores=character_ability_scores,
                    attacks = character_attacks)

players = [galdric, larry, curley, moe];

In [27]:
def roll_initiative(actors):
    for actor in actors:
        actor.initiative = roll_dice(1, 20, Modifier[actor.ability_scores.dexterity] + actor.initiative_modifier) + actor.ability_scores.dexterity/100

In [28]:
def sort_actors(actor):
  return actor.initiative

In [42]:
def get_names(actors):
    names = []
    for actor in actors:
        names.append(actor.name)
    return names

In [44]:
def simulate_combat(monsters, players):
    combat = Combat(monsters, players)
    
    monsters = copy.deepcopy(monsters)
    players = copy.deepcopy(players)
    
    actors = monsters + players
    roll_initiative(actors)
    actors.sort(key=sort_actors, reverse=True)
    
    combat.round = 1;
    battle_over = False
    while not battle_over:
        logging.info(f"Round {combat.round} .. FIGHT!!")
        for actor in actors:
                logging.debug(f"{actor.name}'s turn")
                # if actor has a target, hit that target again
                # if actor does not have a target, select a target randomly
                if (actor.targets == [] ):
                    logging.debug(f"{actor.name} needs targets")
                    if (actor.role == Actor.PLAYER):
                        targets = copy.deepcopy(monsters)
                    else:
                        targets = copy.deepcopy(players)
                    # need to select N targets
                    number_targets = 1
                    for attack in actor.attacks:
                        if (attack.number_targets > number_targets):
                            number_targets = attack.number_targets
                    targeted = []
                    # get N targets, removing target from list when selected
                    for n in range(number_targets):
                        target = targets.pop(random.randrange(len(targets)))
                        targeted.append(target)
                    actor.targets = targeted
                logging.debug(f"{actor.name} is targeting {get_names(actor.targets)}")
                
                # attack the targets
                # remove target from list if dead, and deselect the target

                
        combat.round = combat.round + 1
        if ( combat.round == 11 ):
            battle_over = True
    logging.info(f"Batle over")
    return combat
    
combat = simulate_combat(monsters, players)
display(combat)

DEBUG roll_dice 1d20+2 = 14
DEBUG roll_dice 1d20+2 = 11
DEBUG roll_dice 1d20+2 = 9
DEBUG roll_dice 1d20+2 = 16
DEBUG roll_dice 1d20+2 = 13
DEBUG roll_dice 1d20+2 = 17
DEBUG roll_dice 1d20+2 = 5
DEBUG roll_dice 1d20+2 = 7
INFO simulate_combat Round 1 .. FIGHT!!
DEBUG simulate_combat Larry's turn
DEBUG simulate_combat Larry is targeting ['goblin1']
DEBUG simulate_combat goblin4's turn
DEBUG simulate_combat goblin4 is targeting ['Galdric']
DEBUG simulate_combat goblin1's turn
DEBUG simulate_combat goblin1 is targeting ['Moe']
DEBUG simulate_combat Galdric's turn
DEBUG simulate_combat Galdric is targeting ['goblin2']
DEBUG simulate_combat goblin2's turn
DEBUG simulate_combat goblin2 is targeting ['Moe']
DEBUG simulate_combat goblin3's turn
DEBUG simulate_combat goblin3 is targeting ['Curley']
DEBUG simulate_combat Moe's turn
DEBUG simulate_combat Moe is targeting ['goblin2']
DEBUG simulate_combat Curley's turn
DEBUG simulate_combat Curley is targeting ['goblin2']
INFO simulate_combat Round

{
    "alive": {
        "monsters": [
            {
                "name": "goblin1",
                "role": "monster",
                "hit_points": 7,
                "armor_class": 15,
                "ability_scores": {
                    "strength": 8,
                    "dexterity": 14,
                    "constitution": 10,
                    "intelligence": 10,
                    "wisdom": 8,
                    "charisma": 8
                },
                "attacks": [
                    {
                        "name": "scimitar",
                        "to_hit_modifier": 4,
                        "damage_dice_type": 6,
                        "number_damage_dice": 1,
                        "damage_modifier": 2,
                        "number_targets": 1,
                        "attack_save_attribute": "none",
                        "attack_save_difficulty": 0,
                        "has_advantage": false
                    }
                ],
         